In [20]:
import numpy as np
import logging
from gen_inst import TSPInstance, load_dataset, dataset_conf
from gls import guided_local_search
from sklearn.preprocessing import StandardScaler
import elkai
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler


SCALE = 1000000
perturbation_moves_map = {
    20: 10,
    50: 10,
    100: 30
}
time_limit_map = {
    20: 0.1,
    50: 0.1,
    100: 1,
}


def heuristics_reevo(distance_matrix: np.ndarray) -> np.ndarray:
    # Normalize the distance matrix
    normalized_distance_matrix = distance_matrix / np.max(distance_matrix)

    # Define factors to control the contribution of different criteria
    weight_distance = 0.6
    weight_promising = 0.4

    # Calculate the reciprocal distance matrix
    reciprocal_distance_matrix = 1 / normalized_distance_matrix

    # Normalize the reciprocal distance matrix
    normalized_reciprocal_distance_matrix = reciprocal_distance_matrix / np.max(reciprocal_distance_matrix)

    # Calculate the promising factor based on the normalized reciprocal distance matrix
    promising_factor = (1 - normalized_reciprocal_distance_matrix) / (distance_matrix.shape[0] - 1)

    # Calculate the mean promise for dynamic thresholding
    mean_promise = np.mean(promising_factor)

    # Calculate the combined promise, considering different factors
    overall_promise = weight_distance * normalized_distance_matrix + weight_promising * promising_factor

    # Sparsify the matrix by setting unpromising elements to zero
    sparsified_matrix = np.where(overall_promise > mean_promise, overall_promise, 0)

    return sparsified_matrix


def calculate_cost(inst: TSPInstance, path: np.ndarray) -> float:
    return inst.distmat[path, np.roll(path, 1)].sum().item()

def solve(inst: TSPInstance) -> float:
    heu = heuristics_reevo(inst.distmat)
    assert tuple(heu.shape) == (inst.n, inst.n)
    heu[heu < 1e-6] = 1e-6
    result = guided_local_search(inst.distmat, heu, perturbation_moves_map[inst.n], time_limit_map[inst.n])
    # print(result)
    return calculate_cost(inst, result)


In [16]:
test_sizes = [20, 50, 100]
dataset_size = 64

# Precompute the optimal solutions
optimal_objs_dict = {}
for problem_size in test_sizes:
    dataset_path = f"dataset/test{problem_size}_dataset.npy"
    dataset = load_dataset(dataset_path)[:dataset_size]
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path} with LKH")

    optimal_objs = []
    for i, instance in tqdm(enumerate(dataset)):
        elkai_dist = elkai.DistanceMatrix(((instance.distmat * SCALE).astype(int)).tolist()) 
        optimal_route = elkai_dist.solve_tsp() # e.g. [0, 2, 1, 0]; with proven optimal solutions up to N=315 (https://github.com/fikisipi/elkai)
        optimal_obj = calculate_cost(instance, np.array(optimal_route[:-1]))
        optimal_objs.append(optimal_obj)
        
    mean_optimal_obj = np.mean(optimal_objs)

    optimal_objs_dict[problem_size] = mean_optimal_obj
    
print(optimal_objs_dict)

64it [00:00, 410.22it/s]
64it [00:02, 29.37it/s]
64it [00:16,  3.92it/s]

{20: 3.8362853943492015, 50: 5.68457994395107, 100: 7.778580370400294}


In [21]:

for problem_size in test_sizes:
    dataset_path = f"dataset/test{problem_size}_dataset.npy"
    dataset = load_dataset(dataset_path)[:dataset_size]
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path}")

    objs = []
    for i, instance in tqdm(enumerate(dataset)):
        obj = solve(instance)
        objs.append(obj)

    mean_obj = np.mean(objs)
    mean_optimal_obj = optimal_objs_dict[problem_size]
    print(f"[*] Average for {problem_size}: {mean_obj:.3f} ({mean_optimal_obj:.3f})")
    print(f"[*] Optimality gap: {(mean_obj - mean_optimal_obj) / mean_optimal_obj * 100:.3f}%")
    print()

64it [00:06,  9.95it/s]


[*] Average for 20: 3.836 (3.836)
[*] Optimality gap: 0.000%



64it [00:06,  9.93it/s]


[*] Average for 50: 5.685 (5.685)
[*] Optimality gap: 0.000%



64it [01:04,  1.00s/it]

[*] Average for 100: 7.779 (7.779)
[*] Optimality gap: 0.000%

